# Limpieza e inspección de los datos
Para organizar nuestro notebook (script), dedicaremos la celda inicial a la carga de librerías necesarias para la ejecución del script completo. Si necesitamos instalar algunas de ellas porque no estén disponibles en nuestro repositorio, las instalaremos y dejaremos indicado que hemos necesitado instalarlas. Esto servirá para la configuración del entorno de trabajo de futuros proyectos. :)

El notebook está esturcturado como sigue:

- Bloque A: carga de datos.
- Bloque B: inspección de datos.
- Bloque C: limpieza y adecuación.
- Bloque D: visualización.
- Bloque E: exportación del conjunto resultante. 

## Librerías

In [ ]:
import pandas as ??

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns
import missingno as msno
from wordcloud import WordCloud, STOPWORDS

from langdetect import detect

import nltk
#nltk.download('stopwords')

In [ ]:
# importar stopwords desde el módulo corpus de nltk
from ???.??? import ???

## BLOQUE A: Carga de los datos

**Pandas**: librería más popular de python  que proporciona las herramientas y estructuras necesarias para manipular y analizar datos.
La estructura básica de Pandas es el **DataFrame**, una colección ordenada de columnas con nombres y tipos, donde una sola fila representa un único caso (observación) y las columnas representan atributos particulares.

Guía: https://pandas.pydata.org/docs/getting_started/index.html

In [ ]:
# Leemos el CSV que contiene los datos y lo cargamos en memoria
df = pd.???('data/Airlines_tweets.csv')

## BLOQUE B: inspección de los datos
El objetivo de la inspección es la familiarización con el conjunto de datos. Algunas preguntas iniciales que podría estar bien hacerse pueden ser:

- ¿En qué tipo de objeto están almacenados los datos? ¿Cuál es su dimensión?
- ¿Hay datos ausentes?
- ¿Que tipos diferentes de datos hay?
- ...

In [ ]:
# Para ver el tipo de objeto con el que estamos trabajando
???(df)

In [ ]:
# Dimensiones del dataframe
df.???

In [ ]:
# Otra manera más interesante de proporcionar la misma información anterior
print(f"Hay un dataframe de {df.shape[???]} filas y {df.shape[???]} columnas.")

In [ ]:
# Resumen conciso del dataframe
???.info()

In [ ]:
# Información descriptiva de las variables numéricas
df.???()

In [ ]:
# Mostrar las primeras 5 filas
df.???()

In [ ]:
# Mostrar las últimas 5 filas
df.???()

In [ ]:
# Gráfico para ver valores faltantes
msno.bar(???)

## BLOQUE C: limpieza y adecuación
Algunos aspectos a tratar en este bloque son:

- Ausencias: eliminaremos las columnas con valores faltantes.
- Elimnaremos también las columnas que no son utiles a nuestro objetivo (tweet_id, tweet_created)
- Texto: limpieza y adecuación del texto

In [ ]:
# Eliminación de columnas/variables que contengan datos ausentes
df = df.???(axis=1) 

In [ ]:
# Comprobación de eliminación de datos ausentes
msno.bar(df)

In [ ]:
# Eliminación de columnas que no son útiles para el posterior modelado.
# Nos quedaremos solo con 'airline_sentiment', 'airline', y 'text'.
df = df.drop(columns=['???', '???','???',
                      '???','???'])

In [ ]:
# Detección del idioma en el que está escrito cada tweet
df['language'] = df['text'].apply(detect)

In [ ]:
# ¿En que idioma están escritos los tweets? ¿Cuantos tweets tenemos para cada idioma?
df['???'].value_counts()

In [ ]:
# Nos quedamos con aquellos que están escritos únicamente en inglés
df = df[df['language']=='???']

In [ ]:
# Comprobamos las dimensiones actuales del dataframe
df.???

## BLOQUE D: visualización
En este bloque utilizaremos las librerias [matplotlib](https://matplotlib.org/) y [seaborn](https://seaborn.pydata.org/) para crear unas sencillas representaciones de los datos a modo general y descriptivo, mientras que para nos ayudaremos de la librería [wordcloud](https://amueller.github.io/word_cloud/) para poder crear visualizaciones acerca de los textos que vamos a analizar.

### Distribución de algunas variables

In [ ]:
# Gráfico de barras para la variable airline_sentiment
sns.countplot(x='airline', palette='viridis', data=df)
plt.title('Distribución de la variable airline_sentiment')
plt.show()

In [ ]:
# Gráfico 'pie' con porcentajes para la variable objetivo airline_sentiment
plt.???(df['airline_sentiment'].value_counts(), autopct="%.2f%%", labels=['???', '???', '???'])
plt.show()

In [ ]:
# Gráfico de barras para la variable airline
sns.countplot(data=???, x='airline', palette='viridis', order=???['airline'].value_counts().index)
plt.title('Número de tweets para cada aerolínea')
plt.show()

Toda la información aquí presentada de modo separado puede ser ensamblada en un solo gráfico. En las siguientes celdas vemos algunos ejemplos:

In [ ]:
# Distribución de 'airline_sentiment' por cada aerolínea. 
# En este caso, utilizamos gráficos que ya están incluidos en el objeto dataframe.
plot = df[['???', '???']].value_counts()
plot.unstack().plot(kind='bar', stacked=True, figsize=(15,10))
plt.???('Distribución de tipo de tweets por aerolínea')
plt.???('Aereolinea')
plt.???('Número de tweets')
plt.show()

### Distibución del número de palabras por tweet

In [ ]:
# Calculamos el número de palabras en cada tweet
df['word_count'] = df['???'].apply(lambda x: len(x.???()))

# Mostrar los primeros 5 tweets con el número de palabras
df[['text','word_count']].???

In [ ]:
# Distribución del número de palabras
df.word_count.???()

In [ ]:
# Histograma de la distribución de palabras por tweet.
# Nuevamente, utilizamos los histogramas proporcionados por el propio dataframe.
df['word_count'].???()

#### Histograma de la distribución del número de caracteres por tweet

In [ ]:
df['text'].str.len().hist()

### Representación de las palabras más comunes
- Consideraremos solo los tweets asociados a un sentimiento positivo o negativo.
- Para cada caso, representaremos gráficamente las palabras más comunes (más apariciones).

In [ ]:
# Filtramos el conjunto de datos para quedarnos solo con los tweets positivos
positivedata = df.loc[df['airline_sentiment']=='???', 'text']

# Hacemos lo mismo esta vez con los tweets negativos
negdata = df.loc[df['airline_sentiment']=='???', 'text']

In [ ]:
# Función para poder realizar el gráfico
def wordcloud_draw(data, color, title):
    words = ' '.join(data)
    wordcloud = WordCloud(stopwords=stopwords.words('english'),
                          background_color=color,
                          width=2500,height=2000).generate(words)
    plt.imshow(wordcloud)
    plt.title(title)
    plt.axis('off')

In [ ]:
# Representamos los dos gráficos en una sola visualización
plt.figure(figsize=[20,10])
plt.subplot(1,2,1)
wordcloud_draw(???,'white','Palabras Positivas más comunes')

plt.subplot(1,2,2)
wordcloud_draw(???, '???','Palabras Negativas más comunes')
plt.show()

## BLOQUE E: Exportación del conjunto resultante
En esta última fase exportamos y guardamos el conjunto de datos ya limpiado, procesado y adecuado. Solo guardaremos aquellas columnas que posteriormente nos serán útiles para crear y entrenar nuestro modelo de NLP.

In [ ]:
# Eliminamos las columnas que ya no serán necesarias para el modelo
df = df.???(columns=['airline', 'language', 'word_count'])

In [ ]:
# Guardamos los datos ya procesados en un fichero csv
df.???('data/dataPrepared.csv', index=False)